## Ingestão

A ingestão foi realizada manualmente utilizando o DBeaver(Cliente SQL), onde importamos os arquivos .CSV de forma manual.
Mas Algumas tabelas a ferramenta não conseguiu importar, por isso usamos o pandas para continuar a ingestão.

In [ ]:
!pip install pandas sqlalchemy psycopg2-binary python-dotenv

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [16]:
import os
from dotenv import load_dotenv

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv(dotenv_path="../.env_dev")


True

In [18]:
def create_conn():
    # Defina os parâmetros de conexão
    db_user = os.getenv('DB_USER')
    db_password = os.getenv('DB_PASSWORD')
    db_host = os.getenv('DB_HOST')
    db_port = '5432'
    db_name = 'magic-steps'

    # Crie a URL de conexão
    connection_url = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

    # Crie um engine do SQLAlchemy
    engine = create_engine(connection_url)
    return engine
engine = create_conn()

In [19]:
tabela_nome = 'merged_data_aluno'
df = pd.read_sql_table(table_name=tabela_nome, con=engine,schema='magic_steps')

df.head()

,IdAlunoRotinaEducacaoInfantil,IdAluno,IdTurma_x,Data,IdProfessorResponsavel,IdUsuarioResponsavel,IdFormularioRotinaEducacaoInfantil,IdAlunoObs,IdTipoOcorrencia,StImpedimento,...,StRecursoLinguaPortuguesaSegundaLingua,StRecursoProvaEmVideo,StPermiteUsoImagem,StEstrangeiro,Unnamed: 55,DataOcorrencia_y,IdTurma,IdSituacaoAlunoTurma_y,IdFaseNotaInicial_y,IdUsuario
0,62995,773,273,2022-06-21 00:00:00,11.0,,1,14875,14,False,...,False,False,False,False,,2021-08-25 13:29:36.773000,116,NaN,,14
1,74713,773,273,2022-08-02 00:00:00,11.0,,1,11986,14,False,...,False,False,False,False,,2021-08-25 13:29:36.773000,116,NaN,,14
2,74713,773,273,2022-08-02 00:00:00,11.0,,1,14875,14,False,...,False,False,False,False,,2021-08-25 13:29:36.773000,116,NaN,,14
3,74733,773,273,2022-08-09 00:00:00,11.0,,1,11986,14,False,...,False,False,False,False,,2021-08-25 13:29:36.773000,116,NaN,,14
4,74733,773,273,2022-08-09 00:00:00,11.0,,1,14875,14,False,...,False,False,False,False,,2021-08-25 13:29:36.773000,116,NaN,,14


In [26]:
df['IdAluno'].value_counts(normalize=True).head(100)

IdAluno
773     0.415816
727     0.303021
1147    0.203386
1174    0.031575
747     0.024445
1310    0.010267
740     0.006111
1267    0.005378
Name: proportion, dtype: float64

# AVISO ⚠️

O arquivo merged_data_aluno esta com veio com problema, pois não traz todos os clientes. Vamos realizar o nosso proprio JOIN das tabelas.

Com isso na pasta de sql encontra-se as queries utilizadas para criar os JOINs e as Views.

In [20]:
# Query para listar todas as tabelas
query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'magic_steps'
"""

df_tables = pd.read_sql_query(query, engine)
print(df_tables)


                    table_name
0       tbsituacaoalunoturma_m
1             tbturmaprofessor
2         tbsituacaoalunoturma
3   tbturmaprofessorsecundario
4                      tbturma
..                         ...
57                 tbmunicipio
58                      tbpais
59                     tbaluno
60                    vw_aluno
61                  tbalunoobs

[62 rows x 1 columns]


In [ ]:
# Leitura dos arquivos enviados
df = pd.read_csv('data/TbAluno.csv')
df_alunoobs = pd.read_csv('data/TbAlunoObs.csv')

In [ ]:
# Salvando algumas tabelas no banco de dados
df.to_sql('tbaluno',engine,'magic_steps',  if_exists='replace', index=False)
df_alunoobs.to_sql('tbalunoobs',engine,'magic_steps',  if_exists='replace', index=False)